## API Tiingo - Diario
###### Helga Zambrana - Abril 2024

#### Importar las librerías necesarias

In [78]:
import requests
from decouple import config
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, DateTime
from sqlalchemy.dialects.postgresql import insert
import datetime

#### Leer el usuario, la contraseña y el token desde la variable de entorno

In [79]:
my_host = config('DATABASE_HOST')
port = config('DATABASE_PORT')
password = config('DATABASE_PASSWORD')
database_name = config('DATABASE_NAME')
user = config('DATABASE_USER')
token = config('TIINGO_TOKEN')

#### Definir la clase ListaSimbolos para manejar los símbolos

In [80]:
class ListaSimbolos:
    def __init__(self, simbolos):
        self.simbolos = simbolos
    
    def __call__(self):
        return self.simbolos

#### Definir la clase TiingoData para manejar los datos de Tiingo

In [81]:
class TiingoData:
    # Inicializar atributos de la clase
    def __init__(self, token):
        self.token = token

    def get_data(self, lista_simbolos):
        simbolos = lista_simbolos()
        precios = []

        for simbolo in simbolos:
            url = f"https://api.tiingo.com/tiingo/daily/{simbolo}/prices"
            parametros = {
                'startDate': '2023-01-01',
                'endDate': datetime.date.today().isoformat(),
                'format': 'json',
            }
            headers = {
                'Authorization': f'Token {self.token}'  # Agregar el token enmascarado en los headers
            }

            try:
                respuesta = requests.get(url, params=parametros, headers=headers)
                respuesta.raise_for_status()  # Lanzar una excepción si la solicitud no es exitosa
                datos = respuesta.json()
                
                # Verificar la estructura de los datos
                if isinstance(datos, list) and all(isinstance(item, dict) for item in datos):
                    for dato in datos:
                        dato['ticker'] = simbolo  # Agregar el símbolo a cada fila porque si no, no lo trae
                    precios.extend(datos)
                else:
                    print(f"Aviso: Los datos recibidos para el símbolo {simbolo} no están en el formato esperado.")
            except requests.exceptions.RequestException as e:
                print(f"Error al hacer la solicitud para el símbolo {simbolo}:", e)

        if precios:
            # Convertir los datos en un DataFrame de pandas
            df = pd.DataFrame(precios)
            # Filtrar y renombrar las columnas necesarias
            columnas = {
                'date': 'fecha',
                'ticker': 'simbolo',
                'open': 'apertura',
                'high': 'maximo',
                'low': 'minimo',
                'close': 'cierre',
                'volume': 'volumen',
            }
            df.rename(columns=columnas, inplace=True)
            # Convertir la columna de fecha a tipo datetime
            df['fecha'] = pd.to_datetime(df['fecha'])
            print("Datos obtenidos exitosamente.")
            return df
        else:
            print("No se obtuvieron datos para los símbolos especificados.")
            return None

#### Conectarse con Amazon Redshift y crear tabla (si no existe)

In [82]:
class BaseDatosRedshift:
    def __init__(self, host, port, database_name, user, password):
        self.host = host
        self.port = port
        self.database_name = database_name
        self.user = user
        self.password = password
        self.conn = None
        self.cur = None
        self.metadata = MetaData()

    def conectar(self):
        try:
            # Conectar a la base de datos utilizando psycopg2
            self.conn = psycopg2.connect(host=self.host, port=self.port, user=self.user, password=self.password, database=self.database_name)
            self.cur = self.conn.cursor()
            print("Conexión a la base de datos exitosa.")
        except Exception as e:
            print(f"Error al crear la conexión a la base de datos:", e)
            
    def tabla_existe(self, nombre_tabla):
        # Verificar si la tabla existe en la base de datos
        self.cur.execute(f"SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = '{nombre_tabla}')")
        return self.cur.fetchone()[0]
    
    def crear_tabla_tiingo(self):
        if self.cur is not None:
            try:
                if not self.tabla_existe('hvzambrana_coderhouse.tiingo'):
                    # Crear la tabla solo si no existe
                    create_table_query = """
                    CREATE TABLE hvzambrana_coderhouse.tiingo (
                        simbolo VARCHAR(10) NOT NULL,
                        fecha TIMESTAMP NOT NULL,
                        apertura FLOAT NULL,
                        maximo FLOAT NULL,
                        minimo FLOAT NULL,
                        cierre FLOAT NULL,
                        volumen INTEGER NULL,
                        fecha_ingesta TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                        PRIMARY KEY (simbolo, fecha)
                    )
                    """
                    self.cur.execute(create_table_query)
                    self.conn.commit()
                    print("Tabla creada exitosamente.")
                else:
                    print("La tabla ya existe.")
            except Exception as e:
                print(f"Error al crear la tabla:", e)
        else:
            print("No hay conexión a la base de datos.")

    def cargar_datos(self, df):
        if self.cur is not None:
            try:
                if not self.tabla_existe('hvzambrana_coderhouse.tiingo'):
                    print("La tabla no existe. No se pueden cargar datos.")
                    return

                # Cargar los datos utilizando psycopg2
                for index, row in df.iterrows():
                    insert_query = """
                    INSERT INTO hvzambrana_coderhouse.tiingo (simbolo, fecha, apertura, maximo, minimo, cierre, volumen, fecha_ingesta)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                    ON CONFLICT (simbolo, fecha) DO UPDATE
                    SET apertura = EXCLUDED.apertura,
                        maximo = EXCLUDED.maximo,
                        minimo = EXCLUDED.minimo,
                        cierre = EXCLUDED.cierre,
                        volumen = EXCLUDED.volumen,
                        fecha_ingesta = EXCLUDED.fecha_ingesta
                    """
                    self.cur.execute(insert_query, (row['simbolo'], row['fecha'], row['apertura'], row['maximo'], row['minimo'], row['cierre'], row['volumen'], datetime.datetime.now()))
                self.conn.commit()
                print("Datos cargados exitosamente.")
            except Exception as e:
                print(f"Error al cargar los datos en la tabla:", e)
        else:
            print("No hay conexión a la base de datos.")

#### Obtener datos de Tiingo y cargarlos en Redshift

In [83]:
# Lista de símbolos a consultar
simbolos = ['AMZN', 'AAPL', 'DIS', 'GOOGL', 'JNJ', 'MCD', 'MELI', 'MSFT', 'NVDA', 'TSLA']
lista_simbolos = ListaSimbolos(simbolos)

# Crear instancia de TiingoData
tiingo_data = TiingoData(token)

# Obtener y mostrar los datos de Tiingo
datos_tiingo_df = tiingo_data.get_data(lista_simbolos)

# Crear instancia de la clase BaseDatosRedshift con los datos de conexión
bd_redshift = BaseDatosRedshift(my_host, port, database_name, user, password)

# Conectar con la base de datos
bd_redshift.conectar()

# Crear tabla en la base de datos
bd_redshift.crear_tabla_tiingo()

# Cargar los datos en Redshift si hay datos disponibles
if datos_tiingo_df is not None:
    bd_redshift.cargar_datos(datos_tiingo_df)
else:
    print("No se obtuvieron datos para los símbolos especificados.")

Error al hacer la solicitud para el símbolo AMZN: 429 Client Error: Too Many Requests for url: https://api.tiingo.com/tiingo/daily/AMZN/prices?startDate=2023-01-01&endDate=2024-04-25&format=json
Error al hacer la solicitud para el símbolo AAPL: 429 Client Error: Too Many Requests for url: https://api.tiingo.com/tiingo/daily/AAPL/prices?startDate=2023-01-01&endDate=2024-04-25&format=json
Error al hacer la solicitud para el símbolo DIS: 429 Client Error: Too Many Requests for url: https://api.tiingo.com/tiingo/daily/DIS/prices?startDate=2023-01-01&endDate=2024-04-25&format=json
Error al hacer la solicitud para el símbolo GOOGL: 429 Client Error: Too Many Requests for url: https://api.tiingo.com/tiingo/daily/GOOGL/prices?startDate=2023-01-01&endDate=2024-04-25&format=json
Error al hacer la solicitud para el símbolo JNJ: 429 Client Error: Too Many Requests for url: https://api.tiingo.com/tiingo/daily/JNJ/prices?startDate=2023-01-01&endDate=2024-04-25&format=json
Error al hacer la solicitud